In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import re
import os
from IPython.core.display import display
pd.set_option('display.max_columns', None,'display.max_rows', None)
pd.set_option('display.max_colwidth', None)
#from ipynb_latex_setup import *
%matplotlib inline
T = 298.15
R = 8.314e-03
RT = R*T
display(RT)
N_avogadro = 6.022140857e+23

2.4788191

In [2]:
def molecular_partition_function(x):
    a = max(x)
    q_i = np.exp(x-a)
    sum_q_i = np.sum(q_i)
    log_q = (a + np.log(sum_q_i))
    return(log_q)

In [3]:
mindex_fixed = [np.array(['H+','H2O']),np.array([1,1])]
dgzero = 'DGZERO'
chg = 'Charge'
isomer = 'Pseudoisomer Index'
fx = 'Mole Fraction'
coeff = 'Stoich. Coeff'

pH = 2.365229276187964
pH = 2.925876215728814

#pH = 5
log_Hconc = -pH*np.log(10)
pH = -log_Hconc/np.log(10)
pka = 16.7/RT/np.log(10)
display(pka)
fixed_metabolite = pd.DataFrame(index=mindex_fixed, columns=[chg, dgzero, fx])
fixed_metabolite.loc[('H+',1),dgzero] = np.float_(RT*log_Hconc)
fixed_metabolite.loc[('H+',1), chg] = np.float_(1.0)
fixed_metabolite.loc[('H+',1),fx] = np.float_(1.0)

2.925876215728814

# Pyruvic acid <=> acetaldehyde + CO2

All free energies of formation in aqueous solution were obtained from http://equilibrator.weizmann.ac.il/, version 2.2 with source code repository commit hash f8bc4ca931f41ae08c5cf15b8945c1b1a85158d0, using the component contribution method ( Noor E, Haraldsdóttir HS, Milo R, Fleming RMT (2013) Consistent Estimation of Gibbs Energy Using Component Contributions. PLoS Comput Biol 9(7): e1003098. https://doi.org/10.1371/journal.pcbi.1003098)

In [4]:
mindex1 = [np.array(['pyruvate','pyruvate','acetaldehyde','CO2']),np.array([1,2,1,1])]
reaction1 = pd.DataFrame(index=mindex1, columns=[chg, dgzero, fx])
reaction1.loc[('pyruvate'), chg] = np.array([0,-1])
#Martell AE, Smith RM, Motekaitis RJ. NIST Critically Selected Stability Constants of Metal Complexes Database, 8.0. NIST: Gaithersburg, MD, 2004
reaction1.loc[('pyruvate',1),dgzero] = np.float_(-485.8) 
reaction1.loc[('pyruvate',2),dgzero] = np.float_(-472.3) + fixed_metabolite.loc[('H+',1),dgzero] 
# Component contribution via eQuilibrator:
reaction1.loc[('pyruvate',1),dgzero] = np.float_(-483.6) 
reaction1.loc[('pyruvate',2),dgzero] = np.float_(-466.9) + fixed_metabolite.loc[('H+',1),dgzero] 
reaction1.loc[('pyruvate'),coeff] = np.array([-1.0,-1.0])

reaction1.loc[('acetaldehyde'), chg] = np.float_(0)
#Alberty RA. Thermodynamics of Biochemical Reactions (1st ed.). Hoboken, New Jersey: John Wiley & Sons
reaction1.loc[('acetaldehyde'),dgzero] = np.float_(-139.0) 
# Component Contribution via eQuilibrator
reaction1.loc[('acetaldehyde'),dgzero] = np.float_(-140.3) 
reaction1.loc[('acetaldehyde'),fx] = np.float_(1.0)
reaction1.loc[('acetaldehyde'),coeff] = np.float_(1.0)

reaction1.loc[('CO2'), chg] = np.float_(0)
#Alberty RA. Thermodynamics of Biochemical Reactions (1st ed.). Hoboken, New Jersey: John Wiley & Sons
reaction1.loc[('CO2'),dgzero] = np.float_(-386.0) 
# Component Contribution:
reaction1.loc[('CO2'),dgzero] = np.float_(-386.0) 
reaction1.loc[('CO2'), fx] = np.float_(1)
reaction1.loc[('CO2'),coeff] = np.float_(1.0)


#ddg = reaction1.loc[('H+',1),dgzero] + reaction1.loc[('pyruvate',2),dgzero] - reaction1.loc[('pyruvate',1),dgzero]
ddg = reaction1.loc[('pyruvate',2),dgzero] - reaction1.loc[('pyruvate',1),dgzero]

display(ddg)

# Calculate molecular partition function for pyruvate
x = np.array([-(reaction1.loc[('pyruvate',1),dgzero]/RT), \
     -((reaction1.loc[('pyruvate',2),dgzero]/RT))]) 
log_q_pyr = molecular_partition_function(x)

#q = (-reaction1.loc[('pyruvate'),dgzero]/RT).array.to_numpy(dtype=float)
#q = np.sum(np.exp(q))
#But mole fraction must be pH dependent:
reaction1.loc[('pyruvate',1),fx] = np.exp(-reaction1.loc[('pyruvate',1),dgzero]/RT - log_q_pyr)                
#reaction1.loc[('pyruvate',2),fx] = np.exp(-(reaction1.loc[('pyruvate',2),dgzero]+reaction1.loc[('H+',1),dgzero])/RT)/q
reaction1.loc[('pyruvate',2),fx] = np.exp(-(reaction1.loc[('pyruvate',2),dgzero]/RT) - log_q_pyr)

display(reaction1)

5.684341886080802e-14

Charge DGZERO Mole Fraction  Stoich. Coeff
pyruvate     1      0 -483.6           0.5           -1.0
             2     -1 -483.6           0.5           -1.0
acetaldehyde 1      0 -140.3             1            1.0
CO2          1      0   -386             1            1.0

Equilibrator results:

pH 7.0   -19.5

pH 2.4   -42.2

pH 0.0.  -42.8

In [5]:
#BBR:
dg0_bbr = np.sum(reaction1[dgzero].multiply(reaction1[fx].multiply(reaction1[coeff]))) 
display(dg0_bbr)
#Alberty
dg0_a = reaction1.loc[('acetaldehyde',1),dgzero]*reaction1.loc[('acetaldehyde',1),coeff] + \
        reaction1.loc[('CO2',1),dgzero]*reaction1.loc[('CO2',1),coeff] + \
        (-1)*RT*log_q_pyr*reaction1.loc[('pyruvate',1),coeff]
display(dg0_a)

-42.69999999999402

-40.98181352971682

# Phosphoenolpyruvate + ADP <=> Pyruvate + ATP

All free energies of formation in aqueous solution were obtained from http://equilibrator.weizmann.ac.il/, version 2.2 with source code repository commit hash f8bc4ca931f41ae08c5cf15b8945c1b1a85158d0, using the component contribution method ( Noor E, Haraldsdóttir HS, Milo R, Fleming RMT (2013) Consistent Estimation of Gibbs Energy Using Component Contributions. PLoS Comput Biol 9(7): e1003098. https://doi.org/10.1371/journal.pcbi.1003098)

In [6]:
mindex2 = [np.array(['PEP','PEP','PEP','PEP','ADP','ADP','ADP','ADP','ADP','ADP','pyruvate','pyruvate',\
                    'ATP','ATP','ATP','ATP','ATP','ATP']),np.array([1,2,3,4,1,2,3,4,5,6,1,2,1,2,3,4,5,6])]
reaction2 = pd.DataFrame(index=mindex2, columns=[chg, dgzero, fx])

reaction2.loc[('PEP'), chg] = np.array([0,-1,-2,-3])
# Component contribution via eQuilibrator:
reaction2.loc[('PEP',1),dgzero] = np.float_(-1324.8) 
reaction2.loc[('PEP',2),dgzero] = np.float_(-1320.5) + fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('PEP',3),dgzero] = np.float_(-1300.2) + 2*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('PEP',4),dgzero] = np.float_(-1265.9) + 3*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('PEP'),coeff] = np.array([-1.0,-1.0,-1.0,-1.0])


reaction2.loc[('ADP'), chg] = np.array([0,-1,-2,-3,-4,-5])
# Component contribution via eQuilibrator:
reaction2.loc[('ADP',1),dgzero] = np.float_(-1992.6) 
reaction2.loc[('ADP',2),dgzero] = np.float_(-1974.3) + fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ADP',3),dgzero] = np.float_(-1945.9) + 2*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ADP',4),dgzero] = np.float_(-1903.5) + 3*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ADP',5),dgzero] = np.float_(-1832.5) + 4*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ADP',6),dgzero] = np.float_(-1752.7) + 5*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ADP'),coeff] = np.array([-1.0,-1.0,-1.0,-1.0,-1.0,-1.0])

reaction2.loc[('ATP'), chg] = np.array([0,-1,-2,-3,-4,-5])
# Component contribution via eQuilibrator:
reaction2.loc[('ATP',1),dgzero] = np.float_(-2875.9) 
reaction2.loc[('ATP',2),dgzero] = np.float_(-2861.5) + fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ATP',3),dgzero] = np.float_(-2842.7) + 2*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ATP',4),dgzero] = np.float_(-2814.2) + 3*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ATP',5),dgzero] = np.float_(-2771.8) + 4*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ATP',6),dgzero] = np.float_(-2700.0) + 5*fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('ATP'),coeff] = np.array([1.0,1.0,1.0,1.0,1.0,1.0])

reaction2.loc[('pyruvate'), chg] = np.array([0,-1])
# Component contribution via eQuilibrator:
reaction2.loc[('pyruvate',1),dgzero] = np.float_(-483.6) 
reaction2.loc[('pyruvate',2),dgzero] = np.float_(-466.9) + fixed_metabolite.loc[('H+',1),dgzero] 
reaction2.loc[('pyruvate'),coeff] = np.array([1.0,1.0])

# Calculate molecular partition function for PEP
x = np.array([-(reaction2.loc[('PEP',i),dgzero]/RT) for i in range(1,5)])
log_q_pep = molecular_partition_function(x)

# Calculate molecular partition function for ADP
x = np.array([-(reaction2.loc[('ADP',i),dgzero]/RT) for i in range(1,7) ])
log_q_adp = molecular_partition_function(x)

# Calculate molecular partition function for ATP
x = np.array([-(reaction2.loc[('ATP',i),dgzero]/RT) for i in range(1,7) ])
log_q_atp = molecular_partition_function(x)

# Calculate molecular partition function for pyruvate
x = np.array([-(reaction2.loc[('pyruvate',i),dgzero]/RT) for i in range(1,3) ])
log_q_pyr = molecular_partition_function(x)

for i in range(1,5):
    reaction2.loc[('PEP',i),fx] = np.exp(-reaction2.loc[('PEP',i),dgzero]/RT - log_q_pep)

for i in range(1,7):
    reaction2.loc[('ADP',i),fx] = np.exp(-reaction2.loc[('ADP',i),dgzero]/RT - log_q_adp)

for i in range(1,7):
    reaction2.loc[('ATP',i),fx] = np.exp(-reaction2.loc[('ATP',i),dgzero]/RT - log_q_atp)

for i in range(1,3):
    reaction2.loc[('pyruvate',i),fx] = np.exp(-reaction2.loc[('pyruvate',i),dgzero]/RT - log_q_pyr)

display(reaction2)

Charge  DGZERO Mole Fraction  Stoich. Coeff
PEP      1      0 -1324.8    0.00541677           -1.0
         2     -1 -1337.2      0.805837           -1.0
         3     -2 -1333.6       0.18859           -1.0
         4     -3   -1316   0.000155583           -1.0
ADP      1      0 -1992.6      0.653983           -1.0
         2     -1   -1991       0.34296           -1.0
         3     -2 -1979.3    0.00305758           -1.0
         4     -3 -1953.6   9.60908e-08           -1.0
         5     -4 -1899.3   2.94581e-17           -1.0
         6     -5 -1836.2   2.59386e-28           -1.0
pyruvate 1      0  -483.6           0.5            1.0
         2     -1  -483.6           0.5            1.0
ATP      1      0 -2875.9      0.216338            1.0
         2     -1 -2878.2      0.547138            1.0
         3     -2 -2876.1      0.234516            1.0
         4     -3 -2864.3    0.00200811            1.0
         5     -4 -2838.6    6.3109e-08            1.0
         6     -5 -2783.5   1.40104e-17            1.0

In [7]:
#BBR:
dg0_bbr = np.sum(reaction2[dgzero].multiply(reaction2[fx].multiply(reaction2[coeff]))) 
display(dg0_bbr)
#Alberty
dg0_a = (-1)*RT*log_q_pep*reaction2.loc[('PEP',1),coeff] + \
        (-1)*RT*log_q_adp*reaction2.loc[('ADP',1),coeff] + \
        (-1)*RT*log_q_pyr*reaction2.loc[('pyruvate',1),coeff] + \
        (-1)*RT*log_q_atp*reaction2.loc[('ATP',1),coeff]
display(dg0_a)

-32.320821116705226

-33.625246057264576